### Considerações Iniciais

Este notebook é uma tradução do notebook [Interaction](https://jjallaire.github.io/visualization-curriculum/altair_interaction.html) do Visualization Curriculum. Devido às atualizações do Altair desde a publicação do notebook original, decidimos alterar algumas partes do texto e do código para mantê-los atualizados e coerentes com a versão atual do Altair (5.5.0).

# <font color="#747a7f">6</font> Interação

<em>"Um gráfico não é 'desenhado' de uma vez; ele é 'construído' e reconstruído até revelar todos os relacionamentos constituídos pelas relações entre os dados. As melhores operações de gráficos são aquelas feitas pelas decisões deles mesmos." — [Jacques Bertin](https://books.google.com/books?id=csqX_xnm4tcC)</em>

A visualização oferece um poderoso método de dar sentido aos dados. Uma única imagem, entretanto, tipicamente oferece respostas para, no melhor dos casos, um punhado de perguntas. Pela *interação* nós podemos transformar imagens estáticas em ferramentas de exploração: destacando pontos de interesse, dando zoom para revelar padrões sutis, e fazer ligações entre visualizações múltiplas para pensar sobre relacionamentos multi-dimensionais.

Nos fundamentos da interação está a noção de uma *seleção*: um método de indicar para o computador quais elementos ou regiões nós estamos interessados. Por exemplo, nós podemos passar o mouse sobre um ponto, clicar em várias marcas, ou desenhar um contorno ao redor de uma região para destacar subconjuntos de dados para investigações mais minuciosas.

Além de codificações visuais e transformações de dados, o Altair oferece uma abstração *selection* para criar interações. Estas seleções encorporam três aspectos:

1. Evento de tratamento de entrada para selecionar pontos ou regiões de interesse, assim como mouse hover, click, drag, scroll, e eventos de toque.
2. Generalizar a partir de uma entrada para formar uma regra de seleção (ou <em>[predicado](https://en.wikipedia.org/wiki/Predicate_%28mathematical_logic%29)</em>) que determina se um dado registro está ou não na seleção.
3. Usar o predicado de seleção para configurar dinamicamente uma visualização por meio de *codificações condicionais*, *transformações de filtro*, ou *domínios de escala*.

Este notebook introduz seleções interativas e explora como usá-las para criar uma variedade de técnicas de interação, assim como consultas dinâmicas, panning & zooming, details-on-demand, e brushing & linking.

Este notebook é parte do [data visualization curriculum](https://github.com/uwdata/visualization-curriculum).

In [4]:
import pandas as pd
import altair as alt

## <font color="#747a7f">6.1</font> Datasets

Nós iremos visualizar uma variedade de datasets da coleção [vega-datasets](https://github.com/vega/vega-datasets):

- Um dataset de carros (`cars`) dos anos 70 e início dos anos 80,
- Um dataset de filmes (`movies`), previamente usados no notebook [Data Transformation](https://github.com/uwdata/visualization-curriculum/blob/master/altair_data_transformation.ipynb),
- Um dataset contendo dez anos dos preços das ações [S&P 500](https://en.wikipedia.org/wiki/S%26P_500_Index) (`sp500`),
- Um dataset de ações (`stocks`) de empresas de tecnologia, e
- Um dataset de voos (`flights`), incluindo tempo de embarque, distância, e atraso no desembarque.

In [5]:
cars = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/cars.json'
movies = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/movies.json'
sp500 = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/sp500.csv'
stocks = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/stocks.csv'
flights = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/flights-5k.json'

## <font color="#747a7f">6.2</font> Introduzindo Seleções

Vamos começar com uma seleção básica: simplesmente clicando um ponto para destacá-lo. Usando o dataset `cars`, nós iremos começar com um scatter plot de cavalos de potência versos milhas por galão, com uma codificação de cor para o número de cilíndros no motor do carro.

Além disso, nós iremos criar uma instância de seleção chamando `alt.selection_point()`, indicando que nós queremos uma seleção definida sobre um único valor. Por padrão, a seleção usa um mouse click para determinar o valor selecionado. Para registrar uma seleção com o gráfico, nós devemos adicioná-lo usando o método `.add_params()`.

Assim que nossa seleção é definida, nós podemos usá-la como um parâmetro para *codificações condicionais*, as quais aplicam uma codificação diferente a depender se um registro está dentro ou fora da seleção. Por exemplo, considere o seguinte código:

In [6]:
color=alt.condition(selection, 'Cylinders:O', alt.value('grey'))

Esta definição de codificação declara que pontos contidos na `selection` devem ser coloridos de acordo ao campo `Cylinder`, enquanto pontos não-selecionados devem usar um cinza padrão. Uma seleção vazia inclue *todos* os pontos, e então inicialmente todos os pontos irão ser coloridos.

*Tente clicar em diferentes pontos do gráfico abaixo com click e shift-ckick. O que acontece? (Clique no fundo para limpar a declaração da seleção e retornar para uma seleção "vazia".)*

In [7]:
selection = alt.selection_point();

alt.Chart(cars).mark_circle().add_params(
    selection
).encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color=alt.condition(selection, 'Cylinders:O', alt.value('grey')),
    opacity=alt.condition(selection, alt.value(0.8), alt.value(0.1))
)

alt.Chart(...)

Como nós iremos ver, seleções de pontos oferecem úteis blocos de construção para interações mais poderosas. Ademais, seleções de pontos são somente um dos dois tipos de seleção oferecidos pelo Altair:

- `selection_point` - seleciona múltiplos valores discretos. O primeiro valor é selecionado no mouse click e valores adicionais alternados usando shift-click.
- `selection_interval` - seleciona um intervalo contínuo de valores, iniciado por mouse drag.

Vamos comparar cada um destes tipos de seleção lado a lado. Para manter nosso código organizado nós iremos definir uma função (`plot`) que gera uma especificação de scatter plot exatamente como a função abaixo. Nós podemos passar uma seleção para a função `plot` para tê-la aplicada ao gráfico:

In [8]:
def plot(selection):
    return alt.Chart(cars).mark_circle().add_params(
        selection
    ).encode(
        x='Horsepower:Q',
        y='Miles_per_Gallon:Q',
        color=alt.condition(selection, 'Cylinders:O', alt.value('grey')),
        opacity=alt.condition(selection, alt.value(0.8), alt.value(0.1))
    ).properties(
        width=240,
        height=180
    )

Vamos usar nossa função `plot` para criar duas variantes de gráficos, um para cada tipo de seleção.

O primeiro gráfico (`point`) replica nosso exemplo anterior, suportando interações shift-click para alternar inclusões de múltiplos pontos na seleção. O segundo gráfico (`interval`) gera uma região de seleção (ou *brush*) no mouse drag. Uma vez criada, você pode arrastar o brush ao redor para selecionar diferentes pontos, ou rolar o scroll quando o cursor está dentro do brush para mudar o tamanho do brush (zoom).

*Tente interagir com cada um dos gráficos abaixo!*

In [9]:
alt.hconcat(
  plot(alt.selection_point()).properties(title='Point (Click e Shift-Click)'),
  plot(alt.selection_interval()).properties(title='Interval (Drag)')
)

alt.HConcatChart(...)

O exemplo abaixo usa interações padrões (click, shift-click, drag). Nós podemos personalizar mais as interações oferecendo especificações de eventos de entrada usando a [sintáxe do seletor de eventos do Vega](https://vega.github.io/vega/docs/event-streams/). Por exemplo, nós podemos modificar nosso gráfico `point` para agir nos eventos `mouseover` ao invés de eventos `click`.

*Segure a tecla shift no gráfico para "pintar" com dados!*

In [10]:
alt.hconcat(
  plot(alt.selection_point(on='mouseover')).properties(title='Point (Mouseover e Shift-Mouseover)'),
)

alt.HConcatChart(...)

Agora que nós cobrimos o básico de seleções do Altair, vamos fazer um tour pelas várias técnicas de interação que ele permite!

## <font color="#747a7f">6.3</font> Consultas Dinâmicas

*Consultas dinâmicas* permitem uma exploração rápida e reversível dos dados para isolar padrões de interesse. Conforme definido por [Ahlberg, Williamson e Shneiderman](https://www.cs.umd.edu/~ben/papers/Ahlberg1992Dynamic.pdf), uma consulta dinâmica:

- representa uma consulta graficamente,
- oferece limites visíveis no intervalo da consulta,
- oferece uma representação gráfica dos dados e do resultado da consulta,
- fornece feedback imediato do resultado após cada ajuste da consulta,
- e permite que usuários iniciantes comecem a trabalhar com pouco treinamento.

Uma abordagem comum é modificar os parâmetros da consulta usando widgets de interface do usuário padrões, como controles deslizantes, botões do tipo radio e menus drop-down. Para gerar widgets de consulta dinâmica, nós podemos aplicar uma operação `bind` da seleção a um ou mais campos que desejamos fazer uma consulta.

Vamos construir um scatter plot interativo que usa uma consulta dinâmica para filtrar a exibição. Dado um scatter plot de classificações de filmes (do Rotten Tomatoes e IMDB), nós podemos adicionar uma seleção sobre o campo `Major_Genre` para permitir uma filtragem interativa por gênero do filme.

Para começar, vamos extrair os gêneros únicos (não nulos) dos dados em `movies`:

In [11]:
df = pd.read_json(movies) # Carrega os dados dos filmes
genres = df['Major_Genre'].unique() # retorna valores de campos únicos
genres = list(filter(lambda d: d is not None, genres)) # filtra valores Nulos (None)
genres.sort() # ordena alfabeticamente

Para uso posterior, vamos também definir uma lista de valores únicos de `MPAA_Rating`:

In [12]:
mpaa = ['G', 'PG', 'PG-13', 'R', 'NC-17', 'Not Rated']

Agora vamos criar um limite de seleção `point` para um menu drop-down.

*Use o menu de consulta dinâmica abaixo para explorar os dados. Como as avaliações variam por gênero? Como você modificaria o código para filtrar por `MPAA_Rating` (G, PG, PG-13, etc.) ao invés de `Major_Genre`?*

In [13]:
selectGenre = alt.selection_point(
    name='Select', # nomeia a seleção como 'Select'
    fields=['Major_Genre'], # limita a seleção ao campo Major_Genre
    value=genres[0], # usa a primeira entrada de gênero como valor inicial
    bind=alt.binding_select(options=genres) # vincula a um menu de valores únicos de gênero
)

alt.Chart(movies).mark_circle().add_params(
    selectGenre
).encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q',
    tooltip='Title:N',
    opacity=alt.condition(selectGenre, alt.value(0.75), alt.value(0.05))
)

alt.Chart(...)

Nossa construção acima aproveita vários aspectos das seleções:

- Nós demos à seleção um nome (`'Select'`). Este nome não é obrigatório, mas nos permite influenciar o texto no rótulo do menu de consulta dinâmica gerado. (*O que acontece se você remover o nome? Teste!*)
- Nós restringimos a seleção a um campo específico (`Major_Genre`). Anteriormente, quando nós usamos uma seleção `point`, a seleção mapeou pontos individuais. Ao limitar a seleção a um campo específico, nós podemos selecionar *todos* os pontos cujo valor no campo `Major_Genre` corresponda ao valor selecionado.
- Nós inicializamos a seleção com um valor inicial usando `value=....`
- Nós vinculamos a seleção, com o `bind`, a um widget de interface, neste caso um menu drop-down usando o `binding_select`.
- Como antes, nós usamos ainda uma codificação condicional para controlar o canal de opacidade.

### <font color="#747a7f">6.3.1</font> Vinculando Seleções a Múltiplas Entradas

Uma única instância de seleção pode ser limite para *múltiplos* widgets de consulta dinâmica. Vamos modificar o exemplo acima para oferecer filtros para ambos, `Major_Genre` e `MPAA_Rating`, usando botões do tipo radio ao invés de um menu. Nossa seleção `point` agora é definida por um único *par* de valores de gênero e classificação MPAA.

*Procure combinações inesperadas de gênero e classificação. Há algum filme de terror classificado como G ou PG?*

In [14]:
# seleção de valor único sobre pares [Major_Genre, MPAA_Rating]
# usa valores específicos predefinidos como os valores iniciais selecionados
selection = alt.selection_point(
    name='Select',
    fields=['Major_Genre', 'MPAA_Rating'],
    value=[{'Major_Genre': 'Drama', 'MPAA_Rating': 'R'}],
    bind={
        'Major_Genre': alt.binding_select(options=genres),
        'MPAA_Rating': alt.binding_radio(options=mpaa)
    }
)

# scatter plot, modifica a opacidade com base na seleção
alt.Chart(movies).mark_circle().add_params(
    selection
).encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q',
    tooltip='Title:N',
    opacity=alt.condition(selection, alt.value(0.75), alt.value(0.05))
)


alt.Chart(...)

*Curiosidades: A classificação PG-13 não existia quando os filmes [Tubarão](https://www.imdb.com/title/tt0073195/) e [Tubarão 2](https://www.imdb.com/title/tt0077766/) foram lançados. O primeiro filme a receber a classificação PG-13 foi [Amanhecer Violento](https://www.imdb.com/title/tt0087985/), de 1984.*

### <font color="#747a7f">6.3.2</font> Usando Visualizações como Consultas Dinâmicas

Embora os widgets de interface padrões mostrem os *possíveis* valores dos parâmetros da consulta, eles não visualizam a *distribuição* desses valores. Nós também podemos desejar usar interações mais ricas, como seleções de múltiplos valores ou de intervalos, ao invés de widgets de entrada que selecionam apenas um valor por vez.

Para resolver estes problemas, nós podemos criar gráficos adicionais para visualizar os dados e suportar consultas dinâmicas. Vamos adicionar um histograma da contagem de filmes por ano e usar uma seleção de intervalo para destacar dinamicamente os filmes em períodos de tempo selecionados.

*Interaja com o histograma de anos para explorar filmes de diferentes períodos. Você vê algum indício de [viés de amostragem](https://en.wikipedia.org/wiki/Sampling_bias) ao longo dos anos? (Como os anos e as classificações dos críticos se relacionam?)*

*Os anos variam de 1930 a 2040! Os filmes futuros estão em pré-produção, ou há erros do tipo "erro por um século"? Além disso, dependendo de qual fuso horário você está, você pode ver um pequeno pico ou em 1969 ou em 1970. Por que isso acontece? (Veja o final do notebook para uma explicação!)*

In [15]:
brush = alt.selection_interval(
    encodings=['x'] # limita a seleção aos valores do eixo x (ano)
)

# histograma de consulta dinâmica
years = alt.Chart(movies).mark_bar().add_params(
    brush
).encode(
    alt.X('year(Release_Date):T', title='Filmes por Ano de Lançamento'),
    alt.Y('count():Q', title=None)
).properties(
    width=650,
    height=50
)

# scatter plot, modifica a opacidade com base na seleção
ratings = alt.Chart(movies).mark_circle().encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q',
    tooltip='Title:N',
    opacity=alt.condition(brush, alt.value(0.75), alt.value(0.05))
).properties(
    width=650,
    height=400
)

alt.vconcat(years, ratings).properties(spacing=5)


alt.VConcatChart(...)

O exemplo acima oferece consultas dinâmicas usando uma *seleção vinculada* entre gráficos:

- Nós criamos uma seleção `interval` (`brush`) e definimos `encodings=['x']` para limitar a seleção apenas ao eixo x, resultando em um intervalo de seleção unidimensional.
- Nós registramos o `brush` com nosso histograma de filmes por ano por meio do `.add_params(brush)`.
- Nós usamos o `brush` em uma codificação condicional para ajustar a opacidade do scatter plot com o parâmetro `opacity`.

Esta técnica de interação de selecionar elementos em um gráfico e ver destaques vinculados em um ou mais outros gráficos é conhecida como [brushing & linking](https://en.wikipedia.org/wiki/Brushing_and_linking).

## <font color="#747a7f">6.4</font> Panning & Zooming

O scatter plot das classificações dos filmes está um pouco congestionado em alguns pontos, o que dificulta a análise das regiões mais densas. Usando as técnicas de interação *panning* (deslocamento) e *zooming* (zoom), nós podemos inspecionar as regiões densas de forma mais detalhada.

Vamos começar pensando em como poderíamos expressar deslocamento e zoom usando seleções do Altair. O que define a "área de visualização" de um gráfico? *Os domínios das escalas dos eixos!*

Nós podemos alterar os domínios das escalas para modificar o intervalo visualizado dos valores dos dados. Para fazer isso interativamente, nós podemos vincular uma seleção `interval` aos domínios das escalas com o código `bind='scales'`. O resultado é que, ao invés de um intervalo brush que nós podemos arrastar e dar zoom, nós podemos arrastar e dar zoom em toda a área do gráfico!

*No gráfico abaixo, clique e arraste para mover a visualização ou use o scroll do mouse para dar zoom (ajustar a escala) na visualização. O que você pode descobrir sobre a precisão dos valores de classificação oferecidos?*

In [16]:
alt.Chart(movies).mark_circle().add_params(
    alt.selection_interval(bind='scales')
).encode(
    x='Rotten_Tomatoes_Rating:Q',
    y=alt.Y('IMDB_Rating:Q', axis=alt.Axis(minExtent=30)), # usa o parâmetro minExtent para estabilizar o título do eixo
    tooltip=['Title:N', 'Release_Date:N', 'IMDB_Rating:Q', 'Rotten_Tomatoes_Rating:Q']
).properties(
    width=600,
    height=400
)

alt.Chart(...)

*Ao dar zoom, nós podemos ver que os valores das classificações têm precisão limitada! As classificações do Rotten Tomatoes são inteiros, enquanto as classificações do IMDB são truncadas para décimos. Como resultado, há sobreposição de pontos mesmo quando damos zoom, com vários filmes compartilhando os mesmos valores de classificação.*

Lendo o código acima, você pode notar o código `alt.Axis(minExtent=30)` no canal de codificação `y`. O parâmetro `minExtent` garante que um espaço mínimo seja reservado para os marcadores e rótulos dos eixos. Por que fazer isso? Quando fazemos o deslocamento e damos zoom, os rótulos dos eixos podem mudar e causar um deslocamento na posição do título do eixo. Ao definir um valor mínimo de extensão, podemos reduzir movimentos que podem causar distração. *Tente alterar o valor de `minExtent`, por exemplo definindo-o como zero, e então dê zoom para ver o que acontece quando rótulos de eixos mais longos entram na visualização.*

O Altair também inclui uma forma abreviada para adicionar deslocamento e zoom a um gráfico. Ao invés de criar diretamente uma seleção, você pode chamar `.interactive()` para fazer o Altair gerar automaticamente uma seleção de intervalo limitada às escalas do gráfico.

In [17]:
alt.Chart(movies).mark_circle().encode(
    x='Rotten_Tomatoes_Rating:Q',
    y=alt.Y('IMDB_Rating:Q', axis=alt.Axis(minExtent=30)), # usa o parâmetro minExtent para estabilizar o título do eixo
    tooltip=['Title:N', 'Release_Date:N', 'IMDB_Rating:Q', 'Rotten_Tomatoes_Rating:Q']
).properties(
    width=600,
    height=400
).interactive()

alt.Chart(...)

Por padrão, as vinculações de escala para seleções incluem os canais de codificação `x` e `y`. E se quisessemos limitar o deslocamento e o zoom a uma única dimensão? Podemos usar `encodings=['x']` para restringir a seleção apenas ao canal `x`:

In [18]:
alt.Chart(movies).mark_circle().encode(
    x='Rotten_Tomatoes_Rating:Q',
    y=alt.Y('IMDB_Rating:Q', axis=alt.Axis(minExtent=30)),  # Usa minExtent para estabilizar o título do eixo
    tooltip=['Title:N', 'Release_Date:N', 'IMDB_Rating:Q', 'Rotten_Tomatoes_Rating:Q']
).add_params(
    alt.selection_interval(bind='scales', encodings=['x'])  # Usando add_params em vez de add_selection
).properties(
    width=600,
    height=400
)

alt.Chart(...)

*Ao dar zoom ao longo de um único eixo, a forma dos dados visualizados pode mudar, afetando potencialmente nossa percepção de relacionamentos nos dados. [Escolher uma proporção adequada](http://vis.stanford.edu/papers/arclength-banking) é uma consideração importante no design de visualizações!*

## <font color="#747a7f">6.5</font> Navegação: Visão geral + Detalhes


Ao mover e dar zoom, ajustamos diretamente a "área de visualização" de um gráfico. A estratégia de navegação relacionada de *visão geral + detalhes*, ao invés disso, usa uma exibição de visão geral para mostrar *tudo* dos dados, enquanto suporta seleções que movimentam e ampliam uma exibição de foco separada.

Abaixo, temos dois gráficos de área que mostram uma década de flutuações de preços para o índice de ações S&amp;P 500. Inicialmente, ambos os gráficos mostram o mesmo intervalo de dados. *Clique e arraste no gráfico de visão geral inferior para atualizar a exibição de foco e examinar intervalos de tempo específicos.*

In [19]:
brush = alt.selection_interval(encodings=['x']);

base = alt.Chart().mark_area().encode(
    alt.X('date:T', title=None),
    alt.Y('price:Q')
).properties(
    width=700
)

alt.vconcat(
    base.encode(alt.X('date:T', title=None, scale=alt.Scale(domain=brush))),
    base.add_params(brush).properties(height=60),
    data=sp500
)

alt.VConcatChart(...)

<!--Unlike our earlier panning &amp; zooming case, here we don't want to bind a selection directly to the scales of a single interactive chart. Instead, we want to bind the selection to a scale domain in _another_ chart. To do so, we update the `x` encoding channel for our focus chart, setting the scale `domain` property to reference our `brush` selection. If no interval is defined (the selection is empty), Altair ignores the brush and uses the underlying data to determine the domain. When a brush interval is created, Altair instead uses that as the scale `domain` for the focus chart.-->

Ao contrário do nosso conjunto panorâmica e zoom, aqui não queremos vincular uma seleção diretamente às escalas de um único gráfico interativo. Para isso, queremos vincular a seleção a um domínio de escala em _outro_ gráfico. Para fazer isso, atualizamos o canal de codificação `x` para nosso gráfico principal, definindo a propriedade de escala `domain` (domínio) para referenciar nossa seleção de `brush` (Pincel). Se nenhum intervalo for definido (seleção vazia), o Altair ignora o pincel e usa os dados subjacentes para determinar o domínio. Quando um intervalo de pincel é criado, o Altair o usa como o `domain` da escala para o gráfico principal.

## <font color="#747a7f">6.6</font> Detalhes sob demanda

Após nós identificarmos pontos de interesse em uma visualização, por vezes nós queremos saber mais sobre eles. *Detalhes sob demanda* refere-se à consultar interativamente por mais informações sobre os valores selecionados. *Tooltips* são um método útil de oferecer detalhes sob demanda. Entretanto, as tooltips normalmente mostram informações apenas para um ponto por vez. Como nós poderíamos mostrar mais?

O scatter plot das classificações de filmes inclui uma série de potenciais valores discrepantes interessantes onde as classificações do *Rotten Tomatoes* e do *IMDB* divergem. Vamos criar um gráfico que nos permita selecionar pontos interativamente e mostrar seus rótulos. Para acionar a consulta de filtro na interação de passar o mouse ou clicar, usaremos o [operador de composição Altair](https://altair-viz.github.io/user_guide/interactions.html#composing-multiple-selections) `|` ("ou").

*Passe o cursor sobre os pontos no scatter plot abaixo para ver um destaque e um rótulo. Clique com a tecla Shift pressionada para tornar as anotações persistentes e visualizar vários rótulos de uma só vez. Quais filmes são amados pelos críticos do Rotten Tomatoes, mas não pelo público em geral no IMDB (ou vice-versa)? Veja se consegue encontrar possíveis erros, onde dois filmes diferentes com o mesmo nome foram acidentalmente combinados!*

In [20]:
hover = alt.selection_point(
    on='mouseover',  # selecionar no mouseover
    nearest=True,    # seleciona o ponto mais próximo do cursor
    empty='none',    # seleção vazia não deve corresponder a nada
    toggle=False     # remove o shift + mouseover
)

click = alt.selection_point(
    empty='none' # seleção vazia não corresponde a nenhum ponto
)

# codificações do scatter plot compartilhadas por todas as marcas
plot = alt.Chart().mark_circle().encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q'
)

# base compartilhada por novas camadas
base = plot.transform_filter(
    hover | click # filtrar os pontos em ambas as direções
)

# camada com pontos do scatter plot, anotação circular, e títulos dos rótulos
alt.layer(
    plot.add_params(hover).add_params(click),
    base.mark_point(size=100, stroke='firebrick', strokeWidth=1),
    base.mark_text(dx=4, dy=-8, align='right', stroke='white', strokeWidth=2).encode(text='Title:N'),
    base.mark_text(dx=4, dy=-8, align='right').encode(text='Title:N'),
    data=movies
).properties(
    width=600,
    height=450
)

alt.LayerChart(...)

O exemplo acima adiciona três novas camadas ao scatter plot: uma anotação circular, texto branco para fornecer um fundo legível, e texto preto mostrando o título de um filme. Além disso, este exemplo usa duas seleções em conjunto:

1. Uma seleção única (`hover`) que inclui `nearest=True` para selecionar automaticamente o ponto mais próximo conforme o cursor se move.
2. Uma seleção múltipla (`click`) para criar seleções persistentes via shift-click.

Ambas as seleções incluem o conjunto `empty='none'` para indicar que nenhum ponto deve ser incluído se uma seleção estiver vazia. Estas seleções são então combinadas em uma único predicado &mdash; o *ou* lógico de `hover` e `click` &mdash; para incluir pontos que residem em *qualquer* seleção. Nós usamos esta condição para filtrar as novas camadas para mostrar anotações e rótulos apenas para pontos selecionados.

Usando seleções e camadas, nós podemos realizar vários designs diferentes para obter detalhes sob demanda! Por exemplo, aqui está uma série temporal em escala logarítmica de preços de ações de tecnologia, anotada com uma diretriz e categorias para a data mais próxima do cursor:

In [21]:
# seleciona um ponto para o qual oferece detalhes sob demanda
label = alt.selection_point(
    encodings=['x'], # limita a seleção ao valor do eixo x
    on='mouseover',  # seleciona em eventos mouseover
    nearest=True,    # seleciona o ponto mais próximo do cursor
    empty='none'     # seleção vazia não inclui nenhum ponto
)

# define nosso gráfico de linha dos preços das ações
base = alt.Chart().mark_line().encode(
    alt.X('date:T'),
    alt.Y('price:Q', scale=alt.Scale(type='log')),
    alt.Color('symbol:N')
)

alt.layer(
    base, # gráfico de linha base

    # adiciona uma marca para servir como linha guia
    alt.Chart().mark_rule(color='#aaa').encode(
        x='date:T'
    ).transform_filter(label),

    # adiciona marcas circulares para pontos de tempo selecionados, oculta pontos não selecionados
    base.mark_circle().encode(
        opacity=alt.condition(label, alt.value(1), alt.value(0))
    ).add_params(label),

    # adiciona texto branco para oferecer um fundo legível para rótulos
    base.mark_text(align='left', dx=5, dy=-5, stroke='white', strokeWidth=2).encode(
        text='price:Q'
    ).transform_filter(label),

    # adiciona rótulos de texto para o preços das ações
    base.mark_text(align='left', dx=5, dy=-5).encode(
        text='price:Q'
    ).transform_filter(label),

    data=stocks
).properties(
    width=700,
    height=400
)

alt.LayerChart(...)

*Colocando em ação o que nós aprendemos até agora: você pode modificar o scatter plot do filme acima (aquele com a consulta dinâmica ao longo dos anos) para incluir uma marca `rule` que mostra a classificação média do IMDB (ou Rotten Tomatoes) para os dados contidos na seleção `interval` do ano?*

## <font color="#747a7f">6.7</font> Brushing & Linking, Revisão

Anteriormente neste notebook nós vimos um exemplo de *brushing & linking*: usando um histograma de consulta dinâmica para destacar pontos em um scatter plot de classificação de filme. Aqui, nós visitaremos alguns exemplos adicionais envolvendo seleções vinculadas.

Voltando ao dataset `cars`, nós podemos usar o operador `repeat` para construir uma [matriz de scatter plot (SPLOM)](https://en.wikipedia.org/wiki/Scatter_plot#Scatterplot_matrices) que mostra associações entre quilometragem, aceleração e cavalos de potência. Nós podemos definir uma seleção  `interval` e incluí-la *dentro* de nossa especificação de scatter plot repetido para permitir seleções vinculadas entre todos os gráficos.

*Clique e arraste em qualquer uma das visualizações abaixo para realizar o brushing & linking!*

In [22]:
brush = alt.selection_interval(
    resolve='global' # resolve todas as seleções para uma única instância global
)

alt.Chart(cars).mark_circle().add_params(
    brush
).encode(
    alt.X(alt.repeat('column'), type='quantitative'),
    alt.Y(alt.repeat('row'), type='quantitative'),
    color=alt.condition(brush, 'Cylinders:O', alt.value('grey')),
    opacity=alt.condition(brush, alt.value(0.8), alt.value(0.1))
).properties(
    width=140,
    height=140
).repeat(
    column=['Acceleration', 'Horsepower', 'Miles_per_Gallon'],
    row=['Miles_per_Gallon', 'Horsepower', 'Acceleration']
)

alt.RepeatChart(...)

Observe acima o uso de `resolve='global'` na seleção `interval`. A configuração padrão `'global'` indica que em todos os gráficos apenas um brush pode estar ativo por vez. Entretanto, em alguns casos, nós poderíamos querer definir brushes em vários gráficos e combinar os resultados. Se nós usarmos `resolve='union'`, a seleção será a *união* de todos os brushes: se um ponto estiver dentro de qualquer brush ele será selecionado. Alternativamente, se usarmos `resolve='intersect'`, a seleção consistirá na *interseção* de todos os brushes: apenas os pontos que estão dentro de todos os brushes serão selecionados.

*Tente definir o parâmetro `resolve` para `'union'` e `'intersect'` e veja como isso muda a lógica da seleção resultante.*

### <font color="#747a7f">6.7.1</font> Filtragem Cruzada

Os exemplos de brushing & linking que nós analisamos até agora utilizam codificações condicionais, por exemplo para alterar valores de opacidade em resposta a uma seleção. Outra opção é usar uma seleção definida em uma visualização para *filtrar* o conteúdo de outra visualização.

Vamos construir uma coleção de histogramas para o dataset `flights`: o `delay` de chegada (o quão cedo ou tarde um voo chega, em minutos), `distance` (a distância percorrida em milhas) e `time` (hora do dia da partida). Nós usaremos o operador `repeat` para criar os histogramas e adicionar uma seleção de intervalo (`interval`) para o eixo `x`, com seleções filtradas pela interseção (brush).

Em particular, cada histograma consistirá em duas camadas: uma camada de fundo cinza e uma camada de primeiro plano azul, com a camada de primeiro plano filtrada pela interseção das seleções brush. O resultado é uma interação de *filtragem cruzada* entre os três gráficos!

*Arraste os intervalos de seleção nos gráficos abaixo. Ao selecionar voos com maiores ou menores atrasos na chegada, como as distribuições de distância e horário reagem?*


In [23]:
brush = alt.selection_interval(
    encodings=['x'],
    resolve='intersect'
);

hist = alt.Chart().mark_bar().encode(
    alt.X(alt.repeat('row'), type='quantitative',
        bin=alt.Bin(maxbins=100, minstep=1), # até 100 intervalos
        axis=alt.Axis(format='d', titleAnchor='start') # formato inteiro, título alinhado à esquerda
    ),
    alt.Y('count():Q', title=None) # sem título no eixo y
)

alt.layer(
    hist.add_params(brush).encode(color=alt.value('lightgrey')),
    hist.transform_filter(brush)
).properties(
    width=900,
    height=100
).repeat(
    row=['delay', 'distance', 'time'],
    data=flights
).transform_calculate(
    delay='datum.delay < 180 ? datum.delay : 180', # fixa atrasos maiores que 3 horas em 180 minutos
    time='hours(datum.date) + minutes(datum.date) / 60' # horas fracionárias
).configure_view(
    stroke='transparent' # sem outline
)

alt.RepeatChart(...)

*Com a filtragem cruzada, você pode observar que voos atrasados têm mais chance de partir em horários mais tardios. Esse fenômeno é familiar para viajantes frequentes: um atraso pode se propagar ao longo do dia, afetando os voos subsequentes da mesma aeronave. Para aumentar as chances de uma chegada pontual, reserve um voo mais cedo!*

A combinação de múltiplas visualizações e seleções interativas pode possibilitar formas valiosas de análise multidimensional, transformando até mesmo histogramas básicos em poderosas ferramentas para fazer perguntas a um dataset!


## <font color="#747a7f">6.8</font> Sumário

Para mais informações sobre as opções de interação suportadas no Altair, por favor consulte a [documentação de seleção interativa do Altair](https://altair-viz.github.io/user_guide/interactions.html). Para detalhes sobre como personalizar manipuladores de eventos, por exemplo para compor várias técnicas de interação ou suportar entrada por toque em dispositivos móveis, consulte a [documentação de seleção do Vega-Lite](https://vega.github.io/vega-lite/docs/selection.html).

Interessado em aprender mais? - A abstração de *seleção* foi introduzida no artigo [Vega-Lite: A Grammar of Interactive Graphics](http://idl.cs.washington.edu/papers/vega-lite/), de Satyanarayan, Moritz, Wongsuphasawat e Heer. - O sistema PRIM-9 (para projeção, rotação, isolamento e mascaramento em até 9 dimensões) é uma das primeiras ferramentas de visualização interativas, desenvolvida no início dos anos 1970 por Fisherkeller, Tukey e Friedman. [Um vídeo demo retro ainda existe!](https://www.youtube.com/watch?v=B7XoW2qiFUA) - O conceito de brushing & linking foi consolidado por Becker, Cleveland e Wilks no artigo de 1987 [Dynamic Graphics for Data Analysis](https://scholar.google.com/scholar?cluster=14817303117298653693). - Para um resumo abrangente das técnicas de interação para visualização, consulte [Interactive Dynamics for Visual Analysis](https://queue.acm.org/detail.cfm?id=2146416) de Heer & Shneiderman. - Finalmente, para um tratado sobre o que torna a interação eficaz, leia o clássico artigo [Direct Manipulation Interfaces](https://scholar.google.com/scholar?cluster=15702972136892195211) de Hutchins, Hollan e Norman.


#### <font color="#747a7f">6.8.0.1</font> Apêndice: Sobre a Representação do Tempo

Anteriormente, nós observamos um pequeno pico no número de filmes em 1969 e 1970. De onde vem esse pico? E por que 1969 *ou* 1970? A resposta vem de uma combinação de dados ausentes e como o seu computador representa o tempo.

Internamente, datas e horários são representados em relação ao [epoch UNIX](https://en.wikipedia.org/wiki/Unix_time), onde o tempo "zero" corresponde à meia-noite de 1º de janeiro de 1970 no horário [UTC](https://en.wikipedia.org/wiki/Coordinated_Universal_Time), que segue o [meridiano principal](https://en.wikipedia.org/wiki/Prime_meridian). Acontece que há alguns filmes com datas de lançamento ausentes (`null`). Esses valores `null` são interpretados como tempo `0`, e, portanto, mapeados para 1º de janeiro de 1970 no horário UTC. Se você mora nas Américas – e, portanto, em fusos horários "anteriores" – esse ponto exato no tempo corresponde a uma hora anterior em 31 de dezembro de 1969 no seu fuso horário local. Por outro lado, se você mora perto ou a leste do meridiano principal, a data no seu fuso horário local será 1º de janeiro de 1970.

A lição? Sempre seja cético com seus dados e tenha em mente que a forma como os dados são representados (seja como datas e horários, números de ponto flutuante, latitudes e longitudes, *etc.*) pode, por vezes, levar a artefatos que impactam a análise!
